# 건강관리 앱 리뷰 텍스트마이닝 기반 디자인 전략 연구: LDA 토픽 모델링을 중심으로
#### 👨‍💻 Author: Gyeongbin Park(a.k.a., Tony Park)
  - 📬 Contact: dev.gbpark@gmail.com
  - 🔗 Github: https://github.com/park-gb/mHealthApp-review-textmining
  - 📝 Blog: https://heytech.tistory.com/
  
Last Updated @2022-06-07

# 패키지 설치

1) 가상환경
- 가상환경 pipenv 사용 시 아래 명령어를 통해 모든 필요 패키지 설치 가능
    - pipenv install

2) KoNLPy Mecab 설치방법
- 명령어
    - bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
- 설치 오류 시 해결방법: https://heytech.tistory.com/395?category=453616

# 패키지 import

In [1]:
import numpy as np
import pandas as pd
import warnings # 경고 메시지 무시
warnings.filterwarnings(action='ignore')
# 한국어 형태소 분석기 중 성능이 가장 우수한 Mecab 사용
from konlpy.tag import Mecab
mecab = Mecab()
from tqdm import tqdm # 작업 프로세스 시각화
import re # 문자열 처리를 위한 정규표현식 패키지
from gensim import corpora # 단어 빈도수 계산 패키지
import gensim # LDA 모델 활용 목적
import pyLDAvis.gensim_models # LDA 시각화용 패키지
from collections import Counter # 단어 등장 횟수 카운트

# 데이터셋 load

In [2]:
dataset_raw = pd.read_excel('./data/dataset_raw.xlsx')
dataset_raw.head()

app                                         review  rating
0  다리 근육 운동 – 4주 프로그램              다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐       1
1  다리 근육 운동 – 4주 프로그램                                              굿       5
2  다리 근육 운동 – 4주 프로그램                                          최고입니디       5
3  다리 근육 운동 – 4주 프로그램  아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂       5
4  다리 근육 운동 – 4주 프로그램                                              ᆞ       5

# 데이터 탐색
- 데이터셋에서 전반적으로 결측치 존재여부, 데이터 타입, 데이터 개수를 확인합니다.

In [3]:
dataset_raw.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


# 데이터 전처리

## 결측치 확인

In [4]:
dataset_raw.isnull().sum()

app       0
review    2
rating    0
dtype: int64

## 결측치 제거

In [5]:
# axis = 0: 결측치 포함한 모든 행 제거
dataset = dataset_raw.dropna(axis = 0)
dataset.isnull().sum()

app       0
review    0
rating    0
dtype: int64

## 분석에서 제외할 앱 리뷰 삭제

In [6]:
# 제외할 앱 리스트 Load
remove_app_list = pd.read_excel('./data/remove_app_list.xlsx')
remove_app_list.head()

app
0      캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1                   만보기 - 걸음 계산기
2  딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3                  타임캐시 – 돈버는 어플
4                    돈버는어플 - 캐시런

In [7]:
for remove_app in remove_app_list['app']:
    try:
        dataset = dataset[dataset['app'] != remove_app]
    except:
        pass

In [8]:
dataset.reset_index(drop = True, inplace=True)

In [9]:
dataset

app  \
0                     다리 근육 운동 – 4주 프로그램   
1                     다리 근육 운동 – 4주 프로그램   
2                     다리 근육 운동 – 4주 프로그램   
3                     다리 근육 운동 – 4주 프로그램   
4                     다리 근육 운동 – 4주 프로그램   
...                                  ...   
264064  코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~   
264065  코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~   
264066  코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~   
264067  코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~   
264068  코인스텝 - 돈버는 만보기, 건강과 캐시를 동시에 챙기자~   

                                                   review  rating  
0                       다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐       1  
1                                                       굿       5  
2                                                   최고입니디       5  
3           아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂       5  
4                                                       ᆞ       5  
...                                                   ...     ...  
264064  설치하고 로그인 하니 HTTP 500 Internal server error 가 뜨...       2  
264065  위젯도 만들어주면 좋은데...... 제가 다른 걷기 위젯으로 사용 하는 앱이 있는데...       3  
264066           이거 erc223 기반이라는데 토큰이 현매 이더리움위에 올라가져 있나요?       5  
264067                                           좋은 만보기네요       5  
264068                                     좋군요~디자인이 예쁘네요~       5  

[264069 rows x 3 columns]

## 전처리용 딕셔너리 Load

### 불용어 사전 리스트
- 사용자의 서비스 경험과는 무관하지만 빈출되는 단어인 불용어는 미리 제거하는 작업이 필요합니다.

In [10]:
stopword_list = pd.read_excel('./data/stopword_list.xlsx')
stopword_list.head()

stopword
0     가까스로
1       가량
2       가령
3       가민
4    가민커넥트

### 데이터 치환 리스트
- 같은 의미로 사용된 여러 데이터를 특정 단어로 통일화하기 위한 데이터셋입니다.
- 본 프로젝트에서는 특정 단어의 빈출여부가 핵심이기 때문에 같은 의미의 단어를 통일화하는 게 중요합니다.

In [11]:
replace_list = pd.read_excel('./data/replace_list.xlsx')
replace_list.head()

before_replacement after_replacement
0                S헬스              삼성헬스
1                LG폰              스마트폰
2             LG V10              스마트폰
3                 G7              스마트폰
4                 GX              그룹운동

### 한 글자 키워드 리스트
- 일반적으로 토큰이 1글자인 경우는 대부분 불용어에 해당하기 때문에 제거합니다.
- 단, 프로젝트 성격에 따라 1글자의 단어가 핵심 키워드가 될 수 있습니다.
- 아래의 리스트는 이처럼 키워드로 생각되는 1글자 단어를 리스트업 하였습니다.

In [12]:
one_char_keyword = pd.read_excel('./data/one_char_list.xlsx')
one_char_keyword.head()

one_char_keyword
0                컵
1                방
2                물
3                돈
4                꿈

## 데이터 치환
- 같은 의미의 단어를 하나의 단어로 통일하는 작업입니다.
- LDA 토픽 모델링은 빈출 어휘를 중심으로 결과를 제공하기 때문에 단어를 통일할 필요가 있습니다.

In [13]:
def replace_word(review):
    for i in range(len(replace_list['before_replacement'])):
        try:
            # 치환할 단어가 있는 경우에만 데이터 치환 수행
            if replace_list['before_replacement'][i] in review:
                review = review.replace(replace_list['before_replacement'][i], replace_list['after_replacement'][i])
        except Exception as e:
            print(f"Error 발생 / 에러명: {e}")
    return review

In [14]:
dataset['review_prep'] = ''
review_replaced_list = []
for review in tqdm(dataset['review']):
    review_replaced = replace_word(str(review)) # 문자열 데이터 변환
    review_replaced_list.append(review_replaced)
dataset['review_prep'] = review_replaced_list
dataset.head()

100%|██████████████████████████████████| 264069/264069 [07:30<00:00, 586.41it/s]


app                                         review  rating  \
0  다리 근육 운동 – 4주 프로그램              다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐       1   
1  다리 근육 운동 – 4주 프로그램                                              굿       5   
2  다리 근육 운동 – 4주 프로그램                                          최고입니디       5   
3  다리 근육 운동 – 4주 프로그램  아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂       5   
4  다리 근육 운동 – 4주 프로그램                                              ᆞ       5   

                                     review_prep  
0              다른 P4P어플과 연동 하면 기존에 있던 스케줄이 싹 사라짐  
1                                              굿  
2                                          최고입니디  
3  아무곳에서나 확인 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂  
4                                              ᆞ

## 한국어 외 텍스트 제거
- 숫자, 특수문자, 영문 등 서비스 경험과 관련된 의미를 추출해 내기 어려운 모든 문자열을 제거합니다.

In [15]:
review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), dataset['review_prep']))
dataset['review_prep'] = review_removed

## 평점 기준 데이터 분리

In [16]:
# 긍정적 리뷰(평점 5점 만점 기준 4, 5점)
review_pos = dataset[(4 == dataset['rating']) | (dataset['rating'] == 5)]['review_prep']
# 부정적 리뷰(평점 5점 만점 기준 1, 2점)
review_neg = dataset[(1 == dataset['rating']) | (dataset['rating'] == 2)]['review_prep']

## 토큰화
- KoNLPy에서 속도 및 토큰화 측면에서 성능이 가장 우수한 Mecab 형태소 분석기 활용
- 명사가 문장 내 맥락을 파악하는 데 핵심 형태소이며 빈출 어휘를 쉽게 파악하기 위해 명사만 추출

In [17]:
review_tokenized_pos = list(map(lambda review: mecab.nouns(review), review_pos))
review_tokenized_neg = list(map(lambda review: mecab.nouns(review), review_neg))

## 불용어 제거

In [18]:
def remove_stopword(tokens):
    review_removed_stopword = []
    for token in tokens:
        # 토큰의 글자 수가 2글자 이상인 경우
        if 1 < len(token):
            # 토큰이 불용어가 아닌 경우만 분석용 리뷰 데이터로 포함
            if token not in list(stopword_list['stopword']):
                review_removed_stopword.append(token)
        # 토큰의 글자 수가 1글자인 경우
        else:
            # 1글자 키워드에 포함되는 경우만 분석용 리뷰 데이터로 포함
            if token in list(one_char_keyword['one_char_keyword']):
                review_removed_stopword.append(token)
    return review_removed_stopword

## 토큰 3 이상 15개 이하인 리뷰만 선별
일반적으로, 리뷰의 길이가 길수록 사용자 경험이나 기술적 문제 등 사용자 의견이 많이 내포되어 있을 가능성이 높습니다. 하지만, 오히려 지나치게 길이가 긴 리뷰는 주제 파악이나 리뷰 내 단어 간의 조합을 활용하여 특징을 추출하는 데 어려움이 있을 수 있습니다. 따라서 본 프로젝트에서는 각 리뷰에서 추출된 명사의 개수가 3개 이상 15개 이하인 리뷰만을 분석에 활용하였습니다.

In [40]:
MIN_TOKEN_NUMBER = 3 # 최소 토큰 개수
MAX_TOKEN_NUMBER = 15 # 최대 토큰 개수

In [36]:
def select_review(review_removed_stopword):
    review_prep = []
    for tokens in review_removed_stopword:
        if MIN_TOKEN_NUMBER <= len(tokens) <= MAX_TOKEN_NUMBER:
            review_prep.append(tokens)
    return review_prep

In [37]:
review_removed_stopword_pos = list(map(lambda tokens : remove_stopword(tokens), review_tokenized_pos))
review_removed_stopword_neg = list(map(lambda tokens : remove_stopword(tokens), review_tokenized_neg))

In [38]:
review_prep_pos = select_review(review_removed_stopword_pos)
review_prep_neg = select_review(review_removed_stopword_neg)

## 평점별 리뷰 개수

In [39]:
review_num_pos = len(review_prep_pos)
review_num_neg = len(review_prep_neg)
review_num_tot = review_num_pos + review_num_neg

print(f"분석한 리뷰 총 개수: {review_num_tot}")
print(f"긍정적 리뷰: {review_num_pos}개({(review_num_pos/review_num_tot)*100:.2f}%)")
print(f"부정적 리뷰: {review_num_neg}개({(review_num_neg/review_num_tot)*100:.2f}%)")

분석한 리뷰 총 개수: 74773
긍정적 리뷰: 63977개(85.56%)
부정적 리뷰: 10796개(14.44%)


# LDA 토픽 모델링

## 하이퍼파라미터 튜닝

In [41]:
NUM_TOPICS = 10 # 토픽 개수는 하이퍼파라미터
# passes: 딥러닝에서 Epoch와 같은 개념으로, 전체 corpus로 모델 학습 횟수 결정
PASSES = 15 

## 모델 학습

In [42]:
def lda_modeling(review_prep):
    # 단어 인코딩 및 빈도수 계산
    dictionary = corpora.Dictionary(review_prep)
    corpus = [dictionary.doc2bow(review) for review in review_prep]
    # LDA 모델 학습
    model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics = NUM_TOPICS, 
                                            id2word = dictionary, 
                                            passes = PASSES)
    return model, corpus, dictionary

## LDA 토픽 모델링 결과 시각화

In [43]:
def lda_visualize(model, corpus, dictionary, RATING):
    pyLDAvis.enable_notebook()
    result_visualized = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    pyLDAvis.display(result_visualized)
    # 시각화 결과 저장
    RESULT_FILE = './result/lda_result_' + RATING + '.html'
    pyLDAvis.save_html(result_visualized, RESULT_FILE)

## 긍정적 리뷰 토픽 모델링

In [44]:
model, corpus, dictionary = lda_modeling(review_prep_pos)
RATING = 'pos'
lda_visualize(model, corpus, dictionary, RATING)

## 부정적 리뷰 토픽 모델링

In [45]:
model, corpus, dictionary = lda_modeling(review_prep_pos)
RATING = 'neg'
lda_visualize(model, corpus, dictionary, RATING)

/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/gbpark/.local/share/virtualenvs/mHealthapp-review-analysis1-rzGFRNeI/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp mo